In [314]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from dateutil import parser
from datetime import datetime

In [315]:
data = pd.read_csv("https://raw.githubusercontent.com/daggeraile/market_clustering/master/raw_data/data.csv")

In [316]:
data.head()

,Unnamed: 0,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,...,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain,Country
0,0,1826,1970,Graduation,Divorced,84835.0,0,0,6/16/14,0,...,6,1,0,0,0,0,0,1,0,SP
1,1,1,1961,Graduation,Single,57091.0,0,0,6/15/14,0,...,7,5,0,0,0,0,1,1,0,CA
2,2,10476,1958,Graduation,Married,67267.0,0,1,5/13/14,0,...,5,2,0,0,0,0,0,0,0,US
3,3,1386,1967,Graduation,Together,32474.0,1,1,5/11/14,0,...,2,7,0,0,0,0,0,0,0,AUS
4,4,5371,1989,Graduation,Single,21474.0,1,0,4/8/14,0,...,2,7,1,0,0,0,0,1,0,SP


In [299]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           2240 non-null   int64  
 1   ID                   2240 non-null   int64  
 2   Year_Birth           2240 non-null   int64  
 3   Education            2240 non-null   object 
 4   Marital_Status       2240 non-null   object 
 5   Income               2216 non-null   float64
 6   Kidhome              2240 non-null   int64  
 7   Teenhome             2240 non-null   int64  
 8   Dt_Customer          2240 non-null   object 
 9   Recency              2240 non-null   int64  
 10  MntWines             2240 non-null   int64  
 11  MntFruits            2240 non-null   int64  
 12  MntMeatProducts      2240 non-null   int64  
 13  MntFishProducts      2240 non-null   int64  
 14  MntSweetProducts     2240 non-null   int64  
 15  MntGoldProds         2240 non-null   i

In [294]:
original_columns = data.columns

In [295]:
empty_columns = data.columns[data.isnull().any()]

In [296]:
# imputing missing data

for column in empty_columns:
    
    if data[column].dtype == 'O':
        imputer = SimpleImputer(strategy='most_frequent')
        data[[column]] = imputer.fit_transform(data[[column]])

    if data[column].dtype == 'int' or data[column].dtype == 'float':
        imputer = SimpleImputer(strategy='mean')
        data[[column]] = imputer.fit_transform(data[[column]])
        
data = pd.DataFrame(data, columns=original_columns)

In [297]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           2240 non-null   int64  
 1   ID                   2240 non-null   int64  
 2   Year_Birth           2240 non-null   int64  
 3   Education            2240 non-null   object 
 4   Marital_Status       2240 non-null   object 
 5   Income               2240 non-null   float64
 6   Kidhome              2240 non-null   int64  
 7   Teenhome             2240 non-null   int64  
 8   Dt_Customer          2240 non-null   object 
 9   Recency              2240 non-null   int64  
 10  MntWines             2240 non-null   int64  
 11  MntFruits            2240 non-null   int64  
 12  MntMeatProducts      2240 non-null   int64  
 13  MntFishProducts      2240 non-null   int64  
 14  MntSweetProducts     2240 non-null   int64  
 15  MntGoldProds         2240 non-null   i

In [212]:
def convert_datetime(feature):
    error_count = 0
    
    for i in range(5):
        try:
            parser.parse(data[feature].iloc[i])
        except ValueError:
            error_count += 1

    if error_count == 0:
        data[feature] = pd.to_datetime(data[feature])
        data[feature] = data[feature].apply(lambda x: (datetime.now() - x).days)
    
    return None

In [255]:
# Converting datetime features
cat_features = data.select_dtypes(include=["object"]).columns.values

for feature in cat_features:
    convert_datetime(feature)

In [282]:
# onehotencoding categorical features
cat_features = data.select_dtypes(include=["object"]).columns.values
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
ohe_df = ohe.fit_transform(data[cat_features])

# assigning column names
temp = ohe.categories_
ohe_cols = []
for i in temp:
    for j in i:
        ohe_cols.append(j)

ohe_df = pd.DataFrame(ohe_df, columns=ohe_cols)
ohe_df

,2n Cycle,Basic,Graduation,Master,PhD,Absurd,Alone,Divorced,Married,Single,...,Widow,YOLO,AUS,CA,GER,IND,ME,SA,SP,US
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2236,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2237,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2238,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [290]:
pd.concat([data, ohe_df], axis=1)

,Unnamed: 0,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,...,Widow,YOLO,AUS,CA,GER,IND,ME,SA,SP,US
0,0,1826,1970,Graduation,Divorced,84835.0,0,0,2791,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,1,1961,Graduation,Single,57091.0,0,0,2792,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,10476,1958,Graduation,Married,67267.0,0,1,2825,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,3,1386,1967,Graduation,Together,32474.0,1,1,2827,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,5371,1989,Graduation,Single,21474.0,1,0,2860,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,2235,10142,1976,PhD,Divorced,66476.0,0,1,3257,99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2236,2236,5263,1977,2n Cycle,Married,31056.0,1,0,3301,99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2237,2237,22,1976,Graduation,Divorced,46310.0,1,0,3351,99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2238,2238,528,1978,Graduation,Married,65819.0,0,0,3355,99,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
